# 10 - Export for App

## Overview
Export slim datasets for Streamlit app.

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import pandas as pd
from f1ts import config, io_flat

## Load

In [2]:
raw_dir = config.paths()['data_raw']
processed_dir = config.paths()['data_processed']
features_dir = config.paths()['data_features']

sessions = io_flat.read_csv(raw_dir / 'sessions.csv')
laps = io_flat.read_parquet(processed_dir / 'laps_processed.parquet')
stints = io_flat.read_parquet(processed_dir / 'stints.parquet')
strategies = io_flat.read_parquet(features_dir / 'strategy_decisions.parquet')

✓ Loaded sessions.csv: 3 rows, 6 cols
✓ Loaded laps_processed.parquet: 2,533 rows, 22 cols
  Dtypes: {'session_key': dtype('O'), 'driver': dtype('O'), 'lap': dtype('int64'), 'lap_time_ms': dtype('int64'), 'sector1_ms': dtype('int64'), 'sector2_ms': dtype('int64'), 'sector3_ms': dtype('int64'), 'compound': dtype('O'), 'tyre_life': dtype('int64'), 'is_pit_lap': dtype('bool'), 'track_status': dtype('O'), 'stint_id': dtype('int64'), 'tyre_age_laps': dtype('int64'), 'air_temp': dtype('float64'), 'track_temp': dtype('float64'), 'humidity': dtype('float64'), 'rainfall': dtype('bool'), 'wind_speed': dtype('float64'), 'event_type': dtype('O'), 'duration_laps': dtype('float64'), 'lap_number': dtype('int64'), 'circuit_name': dtype('O')}
✓ Loaded stints.parquet: 120 rows, 14 cols
  Dtypes: {'session_key': dtype('O'), 'driver': dtype('O'), 'stint_id': dtype('int64'), 'compound': dtype('O'), 'start_lap': dtype('int64'), 'end_lap': dtype('int64'), 'n_laps': dtype('int64'), 'avg_lap_time_ms': dtype('f

## Transform: Create App Exports

In [3]:
app_export_dir = features_dir / 'app_export'
app_export_dir.mkdir(exist_ok=True)

# Race index
race_index = sessions[['session_key', 'circuit_name', 'date']].copy()
io_flat.write_csv(race_index, app_export_dir / 'race_index.csv')

# Export per-race files (sample for first race)
for session_key in sessions['session_key'].unique():
    session_laps = laps[laps['session_key'] == session_key]
    session_stints = stints[stints['session_key'] == session_key]
    
    if len(session_laps) > 0:
        # Slim down columns for app
        lap_cols = ['session_key', 'driver', 'lap', 'lap_time_ms', 'compound', 'stint_id', 'tyre_age_laps']
        available_cols = [c for c in lap_cols if c in session_laps.columns]
        io_flat.write_parquet(
            session_laps[available_cols],
            app_export_dir / f'{session_key}_laps.parquet'
        )

print(f'✓ Exported data for {len(sessions)} races')

✓ Saved race_index.csv: 3 rows, 3 cols
✓ Saved 2023_1_R_laps.parquet: 947 rows, 7 cols
✓ Saved 2023_2_R_laps.parquet: 835 rows, 7 cols
✓ Saved 2023_3_R_laps.parquet: 751 rows, 7 cols
✓ Exported data for 3 races


## Save: Already saved above

## Repro Notes

- Exported slim datasets for Streamlit
- Created race index and per-race files